In [1]:
import pandas as pd 

In [2]:
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')
customers = pd.read_csv('data/customers.csv')
regions = pd.read_csv('data/regions.csv')

In [3]:
products.Type.unique()

array(['ADVENTURING EQUIPMENT', 'TOOLS & KITS', 'MUSICAL INSTRUMENT',
       'POTIONS & SCROLLS', 'ARMS & ARMOUR', 'ANIMALS & TRANSPORTATION',
       'JEWELRY', 'SUMMONING DEVICE'], dtype=object)

In [4]:
customers['Key Account'].unique()

array(['Private Buyer', 'No Key Account', 'Bardic College Union',
       'House of Wonders', 'Nobility', "Sprite's & Spirits", 'Zhentarim',
       'Black Phoenix', "Bottle's Up", 'Clothes Contact',
       'Lionshield Coster', "Night's Knights", 'Order of the Gauntlet',
       'Three Wishes'], dtype=object)

In [5]:
# Splitting Orders Table by Products

In [6]:
orders[['Products_split']] = orders[['Products']].apply(lambda x: x.str.split(';'))
orders[['productsIDs_split', 'Quantities_splits', 'ProductPricesInCP_split']] = orders[['productsIDs', 'Quantities', 'ProductPricesInCP']].apply(lambda x: x.str.split(','))

orders = orders.explode(['Products_split','productsIDs_split', 'Quantities_splits', 'ProductPricesInCP_split'])
orders.drop(['Products','productsIDs', 'Quantities', 'ProductPricesInCP'],axis=1,inplace=True)

orders[['productsIDs_split', 'Quantities_splits']] = orders[['productsIDs_split', 'Quantities_splits']] .astype(int)
orders['ProductPricesInCP_split'] = orders['ProductPricesInCP_split'] .astype(float)
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
orders['DeliveryDate'] = pd.to_datetime(orders['DeliveryDate'])

In [7]:
# Join all other tables

# Join customer
combine = orders.merge(products, left_on ='productsIDs_split', right_on = 'product_code', how = 'left')\
                .merge(customers.drop(['Territory'],axis=1), left_on ='CustomerID', right_on = 'Account Code', how = 'left')\
                .merge(regions, on = 'Territory', how = 'left')

In [8]:
combine.shape

(878427, 30)

In [9]:
pd.set_option('display.max_columns', None)
combine.head()

,OrderID,OrderDate,DeliveryDate,CustomerID,Territory,CartPriceInCP,CartPrice,Products_split,productsIDs_split,Quantities_splits,ProductPricesInCP_split,product_code,Business Unit,Type,Products Business Line Leader,Subtype,Product Name,Brand Name,Brand Manager,Account Code,Account Type,Key Account,Key Account Manager,Account Name,Account Manager,Nation,Region,Regional Manager,Area,Area Manager
0,1,2019-01-01,2019-01-11,54295995,Bloodstone (Damara),10285,102 GP; 8 SP; 5 CP,Blood of the Lycanthrope Antidote,12436254,8,1128.0,12436254,ADVENTURING,POTIONS & SCROLLS,Bruno Notlot,POTIONS & ALCHEMY,Blood of the Lycanthrope Antidote,MYTHAL,Arentian Mythrandius,54295995,No Key Account,No Key Account,No Key Account Manager,Capucine's Steel Bloodstone (Damara),Lauren Masiak,Damara,Bloodlands Empire,Magdalena Fracci,North,Christine Moore
1,1,2019-01-01,2019-01-11,54295995,Bloodstone (Damara),10285,102 GP; 8 SP; 5 CP,Blanket,24180928,4,2728.0,24180928,ADVENTURING,ADVENTURING EQUIPMENT,Bubbles,SURVIVAL & WILDERNESS,Blanket,U.N.N.,Ruralion Baeneduren,54295995,No Key Account,No Key Account,No Key Account Manager,Capucine's Steel Bloodstone (Damara),Lauren Masiak,Damara,Bloodlands Empire,Magdalena Fracci,North,Christine Moore
2,1,2019-01-01,2019-01-11,54295995,Bloodstone (Damara),10285,102 GP; 8 SP; 5 CP,Robe of Serpents,97076903,9,5517.0,97076903,ADVENTURING,ADVENTURING EQUIPMENT,Bubbles,CLOAK,Robe of Serpents,U.N.N.,Ruralion Baeneduren,54295995,No Key Account,No Key Account,No Key Account Manager,Capucine's Steel Bloodstone (Damara),Lauren Masiak,Damara,Bloodlands Empire,Magdalena Fracci,North,Christine Moore
3,1,2019-01-01,2019-01-11,54295995,Bloodstone (Damara),10285,102 GP; 8 SP; 5 CP,"Mirror, steel",33879927,3,912.0,33879927,ADVENTURING,TOOLS & KITS,Kay Glimshine,TOOLS,"Mirror, steel",RED CURTAIN,Siras Invictus,54295995,No Key Account,No Key Account,No Key Account Manager,Capucine's Steel Bloodstone (Damara),Lauren Masiak,Damara,Bloodlands Empire,Magdalena Fracci,North,Christine Moore
4,2,2019-02-03,2019-02-05,30949079,Arrabar,1196,11 GP; 9 SP; 6 CP,Sword of Life Stealing,43737386,1,91.0,43737386,LUXURY SPECIALTIES,ARMS & ARMOUR,T'avin T'soban,MARTIAL MELEE WEAPON,Sword of Life Stealing,ARCANTIC,Enthoril Murkywaters,30949079,Key Account,Clothes Contact,Asuma Bahamut,Clothes Contact Arrabar,NaN,Chondath,Calim Empire,Mitchell Brown,South,Cipriano Seidel


In [10]:
combine.to_csv('orders_merge.csv',index=False)
combine.to_json('orders_merge.json', orient='records',index=False)